<a href="https://colab.research.google.com/github/hidt4/python-compchem-book/blob/main/compchem_book_ch06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 6章 分子の構造を最適化してみよう


### 環境構築

#### Google Colab上にPsi4をインストール
2023年8月時点では問題なく動作していましたが
今後のColabの開発動向次第では動作しなくなる可能性があります

In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y -c psi4 psi4 python=3.10
import sys
sys.path.append('/usr/local/lib/python3.10/site-packages')

--2023-07-08 00:01:07--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2023-07-08 00:01:07--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73134376 (70M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  69.75M   307MB/s    in 0.2s    

2023-07-08 00:01:07 (307 MB/s) - ‘Miniconda3-latest-Linux

In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import psi4

print(f'current time: {datetime.datetime.now()}')
print(f'python version:\n{sys.version}')
print(f'numpy version: {np.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'psi4 version: {psi4.__version__}')

current time: 2023-07-08 00:03:22.103860
python version:
3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]
numpy version: 1.22.4
pandas version: 1.5.3
psi4 version: 1.7


#### 必要なライブラリと関数の定義

In [ ]:
# 三次元構造可視化のためのライブラリ
!pip install py3Dmol
import py3Dmol


def show_3D(mol: psi4.core.Molecule) -> py3Dmol.view:
    """
    Psi4のMoleculeオブジェクトをpy3Dmolで描画する
    Args:
        mol: 描画対象の分子

    Returns:
        py3Dmol.view: py3Dmolの描画オブジェクト

    """
    view = py3Dmol.view(width=400, height=400)
    xyz = mol.save_string_xyz_file()
    view.addModel(xyz, 'xyz')
    view.setStyle({'stick': {}})
    view.setBackgroundColor('#e1e1e1')
    view.zoomTo()

    return view.show()

In [ ]:
# 振動数可視化のためのライブラリ
!pip install py3Dmol
import py3Dmol

!git clone https://github.com/duerrsimon/normal-mode-jupyter.git
sys.path.append('/content/normal-mode-jupyter')
from helpers import show_normal_modes

Cloning into 'normal-mode-jupyter'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 23 (delta 8), reused 17 (delta 4), pack-reused 0
Unpacking objects: 100% (23/23), 801.95 KiB | 9.55 MiB/s, done.


#### 計算資源の設定

In [ ]:
# 計算資源の確認（CPU, RAM）
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 8
core id		: 0
cpu cores	: 4
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
addres

In [ ]:
!cat /proc/meminfo

MemTotal:       53474568 kB
MemFree:        40834928 kB
MemAvailable:   51900540 kB
Buffers:          211292 kB
Cached:         11054148 kB
SwapCached:            0 kB
Active:          1183848 kB
Inactive:       10717012 kB
Active(anon):       1792 kB
Inactive(anon):   637212 kB
Active(file):    1182056 kB
Inactive(file): 10079800 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:            160380 kB
Writeback:            24 kB
AnonPages:        627348 kB
Mapped:           454124 kB
Shmem:              2928 kB
KReclaimable:     432516 kB
Slab:             513412 kB
SReclaimable:     432516 kB
SUnreclaim:        80896 kB
KernelStack:        6160 kB
PageTables:        13552 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    26737284 kB
Committed_AS:    2997564 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       10128 kB
VmallocChunk:          0 kB
Percpu:          

In [ ]:
n_cpu = os.cpu_count()
ram = os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES') / (1024 ** 3)

In [ ]:
# 環境に応じて計算資源を設定
psi4.set_num_threads(n_cpu)
psi4.set_memory(f'{ram * 0.9: .0f}GB')

46000000000

### 6.1 分子の構造を最適化するとはどういうことだろうか

#### 水分子の計算

In [ ]:
psi4.set_output_file('h2o_opt_hf-sto3g.log')

PosixPath('h2o_opt_hf-sto3g.log')

In [ ]:
# 水分子の構造の設定
h2o = psi4.geometry('''
0 1
O       -0.7520847362      0.3573098626      0.0156794168
H        0.2372416200      0.3907947487     -0.0110698848
H       -1.0414501312      1.0972341870     -0.5754069255
''')

# 構造最適化計算の実行
psi4.optimize('hf/sto-3g', molecule=h2o)

INFO:psi4.optking.optwrapper:Creating a UserComputer
INFO:psi4.optking.optwrapper:
    			-----------------------------------------

    			 OPTKING 3.0: for geometry optimizations 

    			     By R.A. King, Bethel University     

    			        with contributions from          

    			    A.V. Copan, J. Cayton, A. Heide      

    			-----------------------------------------

    
INFO:psi4.optking.optwrapper:
		 -- Optimization Parameters --
	accept_symmetry_breaking       =           False
	alg_geom_maxiter               =              50
	bt_dx_conv                     =           1e-07
	bt_dx_rms_change_conv          =           1e-12
	bt_max_iter                    =              25
	cart_hess_read                 =           False
	consecutive_backsteps_allowed  =               0
	conv_max_DE                    =           1e-06
	conv_max_disp                  =          0.0012
	conv_max_force                 =          0.0003
	conv_rms_disp                  =              -1

Optimizer: Optimization complete!


-74.96599011056438

In [ ]:
# 最適化構造の出力
print(h2o.save_string_xyz())

0 1
 O    0.000000000000    0.000000000000   -0.071153222146
 H    0.758023512299    0.000000000000    0.564626634948
 H   -0.758023512299   -0.000000000000    0.564626634948



In [ ]:
# 最適化構造の描画
show_3D(h2o)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

#### 異なる終了判定基準で計算

In [ ]:
psi4.set_output_file('h2o_opt_hf-sto3g_2.log')
psi4.set_options({'g_convergence': 'GAU_VERYTIGHT'})
psi4.optimize('hf/sto-3g', molecule=h2o)

INFO:psi4.optking.optwrapper:Creating a UserComputer
INFO:psi4.optking.optwrapper:
    			-----------------------------------------

    			 OPTKING 3.0: for geometry optimizations 

    			     By R.A. King, Bethel University     

    			        with contributions from          

    			    A.V. Copan, J. Cayton, A. Heide      

    			-----------------------------------------

    
INFO:psi4.optking.optwrapper:
		 -- Optimization Parameters --
	accept_symmetry_breaking       =           False
	alg_geom_maxiter               =              50
	bt_dx_conv                     =           1e-07
	bt_dx_rms_change_conv          =           1e-12
	bt_max_iter                    =              25
	cart_hess_read                 =           False
	consecutive_backsteps_allowed  =               0
	conv_max_DE                    =              -1
	conv_max_disp                  =           6e-06
	conv_max_force                 =           2e-06
	conv_rms_disp                  =           4e-06

Optimizer: Optimization complete!


-74.96599012250903

In [ ]:
print(h2o.save_string_xyz())

0 1
 O    0.000000000000    0.000000000000   -0.071157472493
 H    0.758086102682    0.000000000000    0.564660362995
 H   -0.758086102682   -0.000000000000    0.564660362995



In [ ]:
psi4.core.clean_options()

### 6.2 局所解と全体最適解の違いを知ろう

#### 1,2-ジクロロエタン

##### ゴーシュ配座

In [ ]:
# ログファイルを指定
psi4.set_output_file('DCE_gauche.log')

PosixPath('DCE_gauche.log')

In [ ]:
# gauche配座の分子を定義
dce_gauche = psi4.geometry('''
0 1
C       -4.7587744478     -0.1541943645     -0.0559642548
C       -3.2346420969     -0.1669054169      0.0013710608
H       -5.1443980180     -1.1552590965      0.2305585920
H       -5.0986095260      0.0746426192     -1.0881859644
Cl      -5.4379603530      1.0500010949      1.0719697870
H       -2.8490185267     -0.9530856920     -0.6813781716
H       -2.8948070187     -0.3957485023      1.0335914176
Cl      -2.5554561917      1.4011538588     -0.5119631531
''')

In [ ]:
# 構造最適化
psi4.optimize('hf/sto-3g', molecule=dce_gauche)

INFO:psi4.optking.optwrapper:Creating a UserComputer
INFO:psi4.optking.optwrapper:
    			-----------------------------------------

    			 OPTKING 3.0: for geometry optimizations 

    			     By R.A. King, Bethel University     

    			        with contributions from          

    			    A.V. Copan, J. Cayton, A. Heide      

    			-----------------------------------------

    
INFO:psi4.optking.optwrapper:
		 -- Optimization Parameters --
	accept_symmetry_breaking       =           False
	alg_geom_maxiter               =              50
	bt_dx_conv                     =           1e-07
	bt_dx_rms_change_conv          =           1e-12
	bt_max_iter                    =              25
	cart_hess_read                 =           False
	consecutive_backsteps_allowed  =               0
	conv_max_DE                    =           1e-06
	conv_max_disp                  =          0.0012
	conv_max_force                 =          0.0003
	conv_rms_disp                  =              -1

Optimizer: Optimization complete!


-986.3008339046556

In [ ]:
print(dce_gauche.save_string_xyz())

0 1
 C    0.037762168477    0.770163230810   -0.971537815281
 C   -0.037762168477   -0.770163230810   -0.971537815281
 H   -0.410203098734    1.151657996993   -1.890882965256
 H    1.071722443363    1.110299182919   -0.905504447358
CL   -0.872199715769    1.495297306073    0.413989076201
 H    0.410203098734   -1.151657996993   -1.890882965256
 H   -1.071722443363   -1.110299182919   -0.905504447358
CL    0.872199715769   -1.495297306073    0.413989076201



In [ ]:
show_3D(dce_gauche)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
# 最適化構造をXYZファイルとして保存
dce_gauche.save_xyz_file('dce_gauche.xyz', True)

##### アンチ配座

In [ ]:
# ログファイルを指定
psi4.set_output_file('DCE_anti.log')

PosixPath('DCE_anti.log')

In [ ]:
# anti配座の分子を定義
dce_anti = psi4.geometry('''
0 1
C       -4.7565813125     -0.0715651139     -0.0312842044
C       -3.2368352322     -0.0032837663      0.0312840387
H       -5.1034202802     -1.0830926821      0.2687419304
H       -5.1034205619      0.1391627311     -1.0651107286
Cl      -5.4702549705      1.1317615051      1.0713658164
Cl      -2.5231615744     -1.2066103447     -1.0713660266
H       -2.8899959804     -0.2140116390      1.0651105567
H       -2.8899962667      1.0082438103     -0.2687420691
''')

In [ ]:
# 構造最適化
psi4.optimize('hf/sto-3g', molecule=dce_anti)

INFO:psi4.optking.optwrapper:Creating a UserComputer
INFO:psi4.optking.optwrapper:
    			-----------------------------------------

    			 OPTKING 3.0: for geometry optimizations 

    			     By R.A. King, Bethel University     

    			        with contributions from          

    			    A.V. Copan, J. Cayton, A. Heide      

    			-----------------------------------------

    
INFO:psi4.optking.optwrapper:
		 -- Optimization Parameters --
	accept_symmetry_breaking       =           False
	alg_geom_maxiter               =              50
	bt_dx_conv                     =           1e-07
	bt_dx_rms_change_conv          =           1e-12
	bt_max_iter                    =              25
	cart_hess_read                 =           False
	consecutive_backsteps_allowed  =               0
	conv_max_DE                    =           1e-06
	conv_max_disp                  =          0.0012
	conv_max_force                 =          0.0003
	conv_rms_disp                  =              -1

Optimizer: Optimization complete!


-986.3030468693084

In [ ]:
print(dce_anti.save_string_xyz())

0 1
 C   -0.058726035953    0.768934050054    0.000000002454
 C    0.058726035953   -0.768934050054    0.000000002454
 H   -0.579756390414    1.121607206969    0.890593560428
 H   -0.579756396161    1.121607207236   -0.890593552285
CL    1.598479166925    1.504167765097   -0.000000001077
CL   -1.598479166925   -1.504167765097   -0.000000001077
 H    0.579756390414   -1.121607206969    0.890593560428
 H    0.579756396161   -1.121607207236   -0.890593552285



In [ ]:
show_3D(dce_anti)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
# 最適化構造をXYZファイルとして保存
dce_anti.save_xyz_file('dce_anti.xyz', True)

#### *N*-メチルアセトアミド

##### トランス型

In [ ]:
# ログファイルを指定
psi4.set_output_file('amide_trans.log')

PosixPath('amide_trans.log')

In [ ]:
# trans型アミドを定義
amide_trans = psi4.geometry('''
0 1
C       -3.7098485198     -0.3873709111      0.0075198021
C       -2.2161209604     -0.5156039410      0.0174313797
H       -4.1174176889     -0.8479074919     -0.9167016561
H       -4.0140291615      0.6808053138      0.0390995026
H       -4.1344285447     -0.9062342937      0.8923386237
O       -1.5258331167      0.4935451068      0.0564569839
N       -1.6406778483     -1.7443753645     -0.0167727078
H       -2.2310225548     -2.5885815756     -0.0495401112
C       -0.2091301745     -1.9100381894     -0.0086522692
H        0.0239171398     -1.3703365067     -0.7563056337
H       -0.1502044449     -1.2104184268      0.9847325102
H       -0.1084673908     -2.4996883295     -0.0623493244
''')

In [ ]:
# 構造最適化計算
psi4.optimize('hf/sto-3g', molecule=amide_trans)

Streaming output truncated to the last 5000 lines.
INFO:psi4.optking.history:	Steps to be used in Hessian update:  15 14 13

INFO:psi4.optking.history:	Updated Hessian (in au) 
 	   0.326236  -0.002998  -0.003339  -0.003015   0.006300   0.005676   0.001689
	   0.008887  -0.002135   0.000929  -0.001894  -0.000704  -0.000422   0.001474
	  -0.000987   0.001482  -0.001023   0.002264  -0.000323  -0.001317  -0.000363
	   0.001115   0.000516   0.005921   0.001051  -0.000742  -0.004285  -0.001420
	  -0.002070  -0.000329  -0.000317   0.000113  -0.001060   0.000906  -0.000156
	  -0.000037  -0.000054   0.000065   0.000100   0.000219  -0.000130  -0.000118
	   0.000220  -0.000953   0.001013
	  -0.002998   0.333484   0.010216   0.007510  -0.014434  -0.008726  -0.003560
	  -0.002112   0.000305  -0.008592   0.004408   0.000819   0.003898  -0.002453
	   0.002203  -0.003538   0.001131  -0.006102   0.000438   0.002557   0.000857
	  -0.004770   0.008372  -0.008502  -0.000937   0.003059   0.000248  -0.0013

Optimizer: Optimization complete!


-243.86163854564126

In [ ]:
print(amide_trans.save_string_xyz())

0 1
 C   -1.893761589177    0.364723963399    0.066800971788
 C   -0.353753130623    0.359100769765   -0.019845417391
 H   -2.315226752520   -0.155580764272   -0.788396791567
 H   -2.254529150606    1.387679244513    0.077987779995
 H   -2.224130653616   -0.136646670982    0.972453111688
 O    0.336911619240    1.362241516357   -0.028467825461
 N    0.203536841557   -0.962404319777   -0.168409275397
 H   -0.343200698965   -1.665808459977    0.342095690261
 C    1.650387442937   -1.086484089699    0.098507337435
 H    2.186111110546   -0.371178580188   -0.522743886970
 H    1.910756650328   -0.896066877975    1.142968769162
 H    1.975067625850   -2.092053762186   -0.164623896218



In [ ]:
show_3D(amide_trans)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
# 最適構造をxyzファイルとして保存
amide_trans.save_xyz_file('amide_trans.xyz', True)

In [ ]:
psi4.freq('hf/sto-3g', molecule=amide_trans)

INFO:psi4.driver.task_planner:PLANNING Atomic:  keywords={'G_CONVERGENCE': 'QCHEM', 'SCF__INTS_TOLERANCE': 1e-12, 'function_kwargs': {}}
INFO:psi4.driver.task_planner:PLANNING Atomic:  keywords={'SCF__D_CONVERGENCE': 1e-08, 'SCF__E_CONVERGENCE': 1e-08, 'G_CONVERGENCE': 'QCHEM', 'SCF__INTS_TOLERANCE': 1e-12, 'function_kwargs': {}}
INFO:psi4.driver.driver:Compute gradient(): method=hf, basis=sto-3g, molecule=default, nre=177.79319239082633
INFO:psi4.driver.driver:Return gradient(): -243.86163854540175
INFO:psi4.driver.driver:[[-0.00000515 -0.00001542  0.00000673]
 [ 0.00001002 -0.00002650 -0.00000377]
 [-0.00000131  0.00000127 -0.00000296]
 [-0.00000303  0.00001978  0.00000167]
 [-0.00000251 -0.00000472 -0.00000457]
 [ 0.00000393  0.00002017 -0.00000284]
 [-0.00003459 -0.00000432  0.00001641]
 [ 0.00002055  0.00002099 -0.00001157]
 [ 0.00001134 -0.00001709 -0.00000911]
 [-0.00000271 -0.00000228 -0.00000308]
 [-0.00000052 -0.00000378  0.00000688]
 [ 0.00000396  0.00001189  0.00000621]]
IN

-243.86163854526498

##### シス型

In [ ]:
# ログファイルを指定
psi4.set_output_file('amide_cis.log')

PosixPath('amide_cis.log')

In [ ]:
# 計算オプションを指定
psi4.set_options({'FULL_HESS_EVERY': 0})

In [ ]:
# cis型アミドを定義
amide_cis = psi4.geometry('''
0 1
C       -1.8393520698      0.5395360054      0.0201416776
C       -0.3360665111      0.5163043932     -0.0431352068
H       -2.2621883973     -0.0970101712     -0.7851531402
H       -2.2211245617      1.5739335314     -0.1177698104
H       -2.1824800965      0.1717236862      1.0098177487
O        0.2533796400      1.5709822841     -0.2272103410
N        0.3883166009     -0.6304019690      0.0963770346
C       -0.1804129128     -1.9455644721      0.3185298065
H        1.4169544822     -0.5706164842      0.0417502268
H       -1.2868042702     -1.9408789470      0.3669547691
H        0.2053650601     -2.3536998162      1.2759269597
H        0.1261793516     -2.6206949891     -0.5074292733
''')

In [ ]:
# 構造最適化計算
psi4.optimize('hf/sto-3g', molecule=amide_cis)

INFO:psi4.optking.optwrapper:Creating a UserComputer
INFO:psi4.optking.optwrapper:
    			-----------------------------------------

    			 OPTKING 3.0: for geometry optimizations 

    			     By R.A. King, Bethel University     

    			        with contributions from          

    			    A.V. Copan, J. Cayton, A. Heide      

    			-----------------------------------------

    
INFO:psi4.optking.optwrapper:
		 -- Optimization Parameters --
	accept_symmetry_breaking       =           False
	alg_geom_maxiter               =              50
	bt_dx_conv                     =           1e-07
	bt_dx_rms_change_conv          =           1e-12
	bt_max_iter                    =              25
	cart_hess_read                 =           False
	consecutive_backsteps_allowed  =               0
	conv_max_DE                    =           1e-06
	conv_max_disp                  =          0.0012
	conv_max_force                 =          0.0003
	conv_rms_disp                  =              -1

Streaming output truncated to the last 5000 lines.
INFO:psi4.driver.driver:[[-0.00103731 -0.00019884  0.00055718]
 [-0.00071820  0.00182289 -0.00462298]
 [-0.00030888  0.00007516 -0.00017381]
 [ 0.00026295  0.00039445  0.00000594]
 [-0.00014478  0.00007074  0.00016000]
 [ 0.00011555  0.00024331  0.00053568]
 [ 0.00218543 -0.00216586  0.00828147]
 [-0.00033463 -0.00003909 -0.00233210]
 [-0.00031419  0.00026311 -0.00235226]
 [ 0.00044918 -0.00007024 -0.00033853]
 [-0.00007966  0.00056459  0.00047294]
 [-0.00007546 -0.00096022 -0.00019353]]
INFO:psi4.optking.history:	Performing BFGS update.
INFO:psi4.optking.history:	Using 4 previous steps for update.
INFO:psi4.optking.history:	Steps to be used in Hessian update:  6 5 4

INFO:psi4.optking.history:	Updated Hessian (in au) 
 	   0.446493   0.005779   0.000781   0.002644   0.040563   0.047607   0.014271
	   0.011891   0.004126   0.006332  -0.003480   0.033532   0.002825   0.022288
	   0.014577   0.009991   0.003373   0.003167  -0.017099  -0.

Optimizer: Optimization complete!


-243.85946291978388

In [ ]:
print(amide_cis.save_string_xyz())

0 1
 C   -1.516298248054    0.527239477059   -0.093918172710
 C    0.024560092983    0.558782638621   -0.111057593120
 H   -1.893596333075   -0.218509308797   -0.787572854926
 H   -1.892620146439    1.501056171290   -0.388990124090
 H   -1.877225660593    0.289850517370    0.902103657908
 O    0.675596157862    1.496113428524   -0.539838146288
 N    0.682445306283   -0.572134027256    0.504292825412
 C    0.130819663376   -1.924606274480    0.276511165571
 H    1.684600453993   -0.540793607720    0.275743820443
 H   -0.865928355094   -1.998326733369    0.706577358782
 H    0.771149105128   -2.645615290022    0.781981433640
 H    0.073508852216   -2.197759070658   -0.780801062821



In [ ]:
show_3D(amide_cis)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
# 最適構造をxyzファイルとして保存
amide_cis.save_xyz_file('amide_cis.xyz', True)

In [ ]:
psi4.freq('hf/sto-3g', molecule=amide_cis)

INFO:psi4.driver.task_planner:PLANNING Atomic:  keywords={'FULL_HESS_EVERY': 0, 'G_CONVERGENCE': 'QCHEM', 'SCF__INTS_TOLERANCE': 1e-12, 'function_kwargs': {}}
INFO:psi4.driver.task_planner:PLANNING Atomic:  keywords={'SCF__D_CONVERGENCE': 1e-08, 'SCF__E_CONVERGENCE': 1e-08, 'FULL_HESS_EVERY': 0, 'G_CONVERGENCE': 'QCHEM', 'SCF__INTS_TOLERANCE': 1e-12, 'function_kwargs': {}}
INFO:psi4.driver.driver:Compute gradient(): method=hf, basis=sto-3g, molecule=default, nre=177.71299227117376
INFO:psi4.driver.driver:Return gradient(): -243.85946291970694
INFO:psi4.driver.driver:[[-0.00001430  0.00000067 -0.00000814]
 [ 0.00000090 -0.00004223  0.00000374]
 [ 0.00000186  0.00000972 -0.00000972]
 [-0.00000497  0.00000107  0.00001099]
 [ 0.00000591 -0.00001230 -0.00000128]
 [ 0.00000490 -0.00001201  0.00000594]
 [ 0.00001758  0.00008410  0.00000787]
 [-0.00002542  0.00000030 -0.00002010]
 [ 0.00001195 -0.00001592 -0.00000200]
 [ 0.00000524 -0.00000396  0.00000520]
 [-0.00000017 -0.00000247  0.00000565

-243.8594629199236

### 6.3 構造最適化のコツについて知ろう

#### 徐々に最適化レベルを上げていく

In [ ]:
psi4.core.clean_options()

In [ ]:
def record_optimization(mol: psi4.core.Molecule, theory: str) -> float:
    """
    分子を指定計算レベルで最適化する時間を計測する
    Args:
        mol: 目的とする分子
        theory: 計算レベル

    Returns:
        float: 最適化にかかった時間

    """
    import time
    start = time.time()
    psi4.optimize(theory, molecule=mol)
    end = time.time()

    return end - start

In [ ]:
# 2つの方法用に同じ初期構造で分子を2つ準備
hcho = psi4.geometry('''
0 1
 C                 -0.70124482    0.39419087    0.00000000
 O                  0.52607218    0.39419087    0.00000000
 H                 -1.29338982    1.33359487    0.00000000
 H                 -1.29338982   -0.54521313    0.00003900
 ''')

hcho2 = hcho.clone()

In [ ]:
psi4.set_output_file('hcho-low-to-high.log')

PosixPath('hcho-low-to-high.log')

In [ ]:
# 計算レベルの設定
level1 = 'mp2/cc-pvdz'
level2 = 'mp2/aug-cc-pvqz'

In [ ]:
# 低レベルでまず最適化してから高精度で最適化
time_1_1 = record_optimization(hcho, level1)
time_1_2 = record_optimization(hcho, level2)
print(hcho.save_string_xyz())
print(f'level 1: {time_1_1: .2f} sec\tlevel 2: {time_1_2: .2f} sec')
print(f'level 1 + 2: {time_1_1 + time_1_2: .2f} sec')
print('####')

INFO:psi4.optking.optwrapper:Creating a UserComputer
INFO:psi4.optking.optwrapper:
    			-----------------------------------------

    			 OPTKING 3.0: for geometry optimizations 

    			     By R.A. King, Bethel University     

    			        with contributions from          

    			    A.V. Copan, J. Cayton, A. Heide      

    			-----------------------------------------

    
INFO:psi4.optking.optwrapper:
		 -- Optimization Parameters --
	accept_symmetry_breaking       =           False
	alg_geom_maxiter               =              50
	bt_dx_conv                     =           1e-07
	bt_dx_rms_change_conv          =           1e-12
	bt_max_iter                    =              25
	cart_hess_read                 =           False
	consecutive_backsteps_allowed  =               0
	conv_max_DE                    =           1e-06
	conv_max_disp                  =          0.0012
	conv_max_force                 =          0.0003
	conv_rms_disp                  =              -1

Optimizer: Optimization complete!


INFO:psi4.optking.optwrapper:Creating a UserComputer
INFO:psi4.optking.optwrapper:
    			-----------------------------------------

    			 OPTKING 3.0: for geometry optimizations 

    			     By R.A. King, Bethel University     

    			        with contributions from          

    			    A.V. Copan, J. Cayton, A. Heide      

    			-----------------------------------------

    
INFO:psi4.optking.optwrapper:
		 -- Optimization Parameters --
	accept_symmetry_breaking       =           False
	alg_geom_maxiter               =              50
	bt_dx_conv                     =           1e-07
	bt_dx_rms_change_conv          =           1e-12
	bt_max_iter                    =              25
	cart_hess_read                 =           False
	consecutive_backsteps_allowed  =               0
	conv_max_DE                    =           1e-06
	conv_max_disp                  =          0.0012
	conv_max_force                 =          0.0003
	conv_rms_disp                  =              -1

Optimizer: Optimization complete!
0 1
 C   -0.000103729826    0.604439881893    0.000000000000
 O    0.000096407027   -0.602269589846    0.000000000000
 H   -0.000147478103    1.180746661733    0.933922523352
 H   -0.000147478103    1.180746661733   -0.933922523352

level 1:  8.14 sec	level 2:  48.83 sec
level 1 + 2:  56.98 sec
####


In [ ]:
# 直接高精度で最適化
time_2 = record_optimization(hcho2, level2)
print(hcho2.save_string_xyz())
print(f'level 2: {time_2: .2f} sec')
print('####')

INFO:psi4.optking.optwrapper:Creating a UserComputer
INFO:psi4.optking.optwrapper:
    			-----------------------------------------

    			 OPTKING 3.0: for geometry optimizations 

    			     By R.A. King, Bethel University     

    			        with contributions from          

    			    A.V. Copan, J. Cayton, A. Heide      

    			-----------------------------------------

    
INFO:psi4.optking.optwrapper:
		 -- Optimization Parameters --
	accept_symmetry_breaking       =           False
	alg_geom_maxiter               =              50
	bt_dx_conv                     =           1e-07
	bt_dx_rms_change_conv          =           1e-12
	bt_max_iter                    =              25
	cart_hess_read                 =           False
	consecutive_backsteps_allowed  =               0
	conv_max_DE                    =           1e-06
	conv_max_disp                  =          0.0012
	conv_max_force                 =          0.0003
	conv_rms_disp                  =              -1

Optimizer: Optimization complete!
0 1
 C   -0.000013375548    0.604462795702    0.000000000000
 O    0.000012987468   -0.602321381403    0.000000000000
 H   -0.000023430089    1.181021231136    0.933558706712
 H   -0.000023430089    1.181021231136   -0.933558706712

level 2:  72.92 sec
####


In [ ]:
# 2つの方法で得た構造の差分を計算
for i in range(hcho.natom()):
    vec1 = hcho.xyz(i)
    vec2 = hcho2.xyz(i)
    dist = vec1.distance(vec2)
    symbol = hcho.symbol(i)
    print(f'{symbol}\t{dist: .3f}')

C	 0.000
O	 0.000
H	 0.001
H	 0.001


#### 一部を固定化して最適化する

##### ジクロロエタン

In [ ]:
dce_constrained = psi4.geometry('''
0 1
 C                 -2.25311207    1.17427384    0.00000000
 H                 -1.69945761    1.74831286   -0.71333438
 H                 -3.29070391    1.18574460   -0.26109690
 C                 -1.73979635   -0.27765831    0.00000000
 H                 -2.29345264   -0.85169798   -0.71333244
 H                 -1.86707706   -0.70095780    0.97443171
 Cl                -2.04374927    1.87054265    1.60280285
 Cl                -0.03310419   -0.29652555   -0.42947162
''')

In [ ]:
show_3D(dce_constrained)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
psi4.core.clean_options()
psi4.p4util.prepare_options_for_set_options()

{}

In [ ]:
psi4.set_output_file('DCE_frozen_dihedral.log')
psi4.set_options({'FROZEN_DIHEDRAL': '7 1 4 8'})
psi4.optimize('hf/sto-3g', molecule=dce_constrained)

INFO:psi4.optking.optwrapper:Creating a UserComputer
INFO:psi4.optking.optwrapper:
    			-----------------------------------------

    			 OPTKING 3.0: for geometry optimizations 

    			     By R.A. King, Bethel University     

    			        with contributions from          

    			    A.V. Copan, J. Cayton, A. Heide      

    			-----------------------------------------

    
INFO:psi4.optking.optwrapper:
		 -- Optimization Parameters --
	accept_symmetry_breaking       =           False
	alg_geom_maxiter               =              50
	bt_dx_conv                     =           1e-07
	bt_dx_rms_change_conv          =           1e-12
	bt_max_iter                    =              25
	cart_hess_read                 =           False
	consecutive_backsteps_allowed  =               0
	conv_max_DE                    =           1e-06
	conv_max_disp                  =          0.0012
	conv_max_force                 =          0.0003
	conv_rms_disp                  =              -1

Optimizer: Optimization complete!


-986.2992649992009

In [ ]:
show_3D(dce_constrained)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
print(dce_constrained.save_string_xyz())

0 1
 C   -0.938414700657    0.488749600149   -0.414000004454
 H   -0.409309673545    1.099477287294   -1.145614302004
 H   -2.004679212409    0.495145997708   -0.645272701544
 C   -0.420455800166   -0.969164268026   -0.418128991461
 H   -0.961710199641   -1.543882426196   -1.171249636652
 H   -0.553551244504   -1.439886831947    0.555988968133
CL   -0.755008973474    1.269842719950    1.209619710473
CL    1.334565959079   -1.064946317862   -0.854717480223



In [ ]:
psi4.p4util.prepare_options_for_set_options()

{'FROZEN_DIHEDRAL': '7 1 4 8', 'SCF__INTS_TOLERANCE': 1e-12}

In [ ]:
psi4.core.clean_options()

In [ ]:
psi4.optimize('hf/sto-3g', molecule=dce_constrained)

INFO:psi4.optking.optwrapper:Creating a UserComputer
INFO:psi4.optking.optwrapper:
    			-----------------------------------------

    			 OPTKING 3.0: for geometry optimizations 

    			     By R.A. King, Bethel University     

    			        with contributions from          

    			    A.V. Copan, J. Cayton, A. Heide      

    			-----------------------------------------

    
INFO:psi4.optking.optwrapper:
		 -- Optimization Parameters --
	accept_symmetry_breaking       =           False
	alg_geom_maxiter               =              50
	bt_dx_conv                     =           1e-07
	bt_dx_rms_change_conv          =           1e-12
	bt_max_iter                    =              25
	cart_hess_read                 =           False
	consecutive_backsteps_allowed  =               0
	conv_max_DE                    =           1e-06
	conv_max_disp                  =          0.0012
	conv_max_force                 =          0.0003
	conv_rms_disp                  =              -1

Optimizer: Optimization complete!


-986.3008338952038

In [ ]:
show_3D(dce_constrained)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
print(dce_constrained.save_string_xyz())

0 1
 C   -1.060741712163    0.442530436272   -0.466328005740
 H   -0.673450909205    1.024871246918   -1.303039513031
 H   -2.151094177391    0.423359275907   -0.513308285493
 C   -0.525746159809   -1.003416892470   -0.505186606960
 H   -0.936607925266   -1.514984556774   -1.377502083706
 H   -0.805307956406   -1.547671229079    0.397494690725
CL   -0.601089398614    1.296508490367    1.061029733972
CL    1.277120961650   -1.057504490347   -0.647049548661



##### ウレア

In [ ]:
psi4.core.clean_options()

In [ ]:
urea = psi4.geometry('''
0 1
C  -1.0482368367  -0.0012423218  -0.7758260561
O  -0.0156753108  -0.0009116043  -1.4283127954
N  -1.7018904431   1.1779245825  -0.3361582586
N  -1.7266400660  -1.1964224417  -0.3989998481
H  -2.6423326976   1.1099656587   0.0477575562
H  -2.6243470466  -1.1132273634   0.0768581166
C  -1.0331079014   2.4327891484  -0.2240686746
C  -1.7709147156   3.6047187295  -0.1240375937
C   0.3646086960   2.4973476556  -0.1752142056
C  -1.1050846099   4.8208129312  -0.0180761207
H  -2.8520156513   3.5729985722  -0.1396804543
C   1.0061712280   3.7072816831  -0.0381144239
H   0.9419000890   1.5862284330  -0.2376082475
C   0.2701868058   4.8845111549   0.0085960416
H   0.7715659747   5.8417509737   0.0568813942
C  -1.0357160633  -2.4408328658  -0.2531541833
C  -1.7721762481  -3.6146113788  -0.1538842773
C   0.3585515106  -2.4973265719  -0.1766237776
C  -1.1060839791  -4.8222638807  -0.0202426056
H  -2.8530368758  -3.5848424540  -0.1873777316
C   1.0045107741  -3.7088549649  -0.0250016816
H   0.9322000371  -1.5820970571  -0.2346267401
C   0.2710808092  -4.8842783600   0.0289372341
H   0.7704073129  -5.8416948859   0.0936212507
C   2.5536477648   3.7773576262  -0.0016853817
C  -1.9225668536   6.1368680035  -0.0023114781
C   2.5550174644  -3.7772234847  -0.0001177715
C  -1.9236673895  -6.1367948329   0.0021592828
F  -1.0990604049   7.2059841426  -0.0034260349
F  -2.7134666506   6.1869873422   1.0987995779
F  -2.7140404800   6.1853874684  -1.1034015072
F   2.9670204746   5.0647872239  -0.0000217451
F   3.0409557981   3.1550722476  -1.1003739505
F   3.0396455561   3.1576990486   1.1017473456
F   3.0403564876  -3.1561078437  -1.0998936671
F   3.0397742797  -3.1567134866   1.1024612903
F   2.9667580606  -5.0647949842   0.0019500673
F  -1.0988989956  -7.2058336061   0.0039753596
F  -2.7133083352  -6.1853211090   1.1043007201
F  -2.7137953394  -6.1873520690  -1.0982323098
''')

In [ ]:
show_3D(urea)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
conditions = ' '.join([f'{id} XYZ' for id in range(25, 41)])
conditions

'25 XYZ 26 XYZ 27 XYZ 28 XYZ 29 XYZ 30 XYZ 31 XYZ 32 XYZ 33 XYZ 34 XYZ 35 XYZ 36 XYZ 37 XYZ 38 XYZ 39 XYZ 40 XYZ'

In [ ]:
psi4.set_output_file('urea_constrained.log')
psi4.set_options({'optking__frozen_cartesian': conditions,
                  'geom_maxiter': 100})
psi4.optimize('hf/sto-3g', molecule=urea)

Streaming output truncated to the last 5000 lines.
	   0.000206   0.001994   0.000489  -0.001927   0.005926  -0.006706   0.001410
	   0.003115  -0.000118   0.001329  -0.003394  -0.000948  -0.003361  -0.001612
	  -0.017322   0.021562   0.003494  -0.009138  -0.001034  -0.000983   0.000699
	   0.000799   0.002012  -0.003250  -0.004564   0.001584  -0.005642  -0.002383
	   0.001002   0.007646   0.018666  -0.029609  -0.000348   0.002245   0.003626
	   0.000235   0.030683  -0.029073  -0.004516   0.005036  -0.001812  -0.000599
	  -0.003705   0.004076   0.002765   0.008768   0.019792  -0.028492   0.029431
	  -0.030306  -0.005779  -0.000319   0.001645   0.000000   0.000000   0.000000
	   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000
	   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000
	   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000
	   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000
	

Optimizer: Optimization complete!


-1998.3932281161635

In [ ]:
print('### Constrained optimization')
psi4.p4util.prepare_options_for_set_options()

### Constrained optimization


{'OPTKING__FROZEN_CARTESIAN': '25 XYZ 26 XYZ 27 XYZ 28 XYZ 29 XYZ 30 XYZ 31 XYZ 32 XYZ 33 XYZ 34 XYZ 35 XYZ 36 XYZ 37 XYZ 38 XYZ 39 XYZ 40 XYZ',
 'GEOM_MAXITER': 100,
 'SCF__INTS_TOLERANCE': 1e-12}

In [ ]:
print(urea.save_string_xyz())

40

 C   -1.066980658044    0.025729115004   -0.424950571785
 O   -0.112355782388    0.095114688063   -1.174861800690
 N   -1.651018771627    1.149453686488    0.273469016295
 N   -1.752514480326   -1.190477425177   -0.111945986235
 H   -2.671528077299    1.200978690648    0.195166491252
 H   -2.451762852586   -1.086537287217    0.629090124789
 C   -0.995591360555    2.421849157608    0.176072349189
 C   -1.734866100922    3.598111410090    0.150757298633
 C    0.398744342809    2.489850866729    0.185064401747
 C   -1.076738945942    4.817527004206    0.118845826771
 H   -2.816664820746    3.561473233928    0.161765697173
 C    1.025561620174    3.716617845584    0.127182130325
 H    0.988233331016    1.585808624573    0.228066559067
 C    0.298368516338    4.890553431357    0.094646608462
 H    0.798492867812    5.848924337670    0.060322062253
 C   -1.010401071290   -2.425861678123   -0.028893908832
 C   -1.739535912049   -3.609856499567    0.022836412988
 C    0.381967348493   -2.4

In [ ]:
show_3D(urea)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
psi4.core.clean_options()

In [ ]:
psi4.optimize('hf/sto-3g', molecule=urea)

Streaming output truncated to the last 5000 lines.
	  -0.002434  -0.001956  -0.002261  -0.002682  -0.002905  -0.001370   0.002468
	   0.000088   0.002628  -0.000592  -0.000644  -0.001335  -0.000481  -0.000804
	   0.000050   0.001927   0.002136  -0.003333  -0.002102  -0.001121  -0.001173
	  -0.000295   0.001068  -0.001461  -0.001267  -0.001946   0.001892   0.000728
	   0.003269   0.001249   0.001088   0.000699  -0.000389   0.000663   0.001895
	   0.000111  -0.000147   0.000420  -0.000471   0.000357  -0.001641  -0.001431
	  -0.001087  -0.000817  -0.000548  -0.000209   0.001785  -0.000038   0.001048
	   0.000887   0.000286  -0.000536  -0.001623   0.000227  -0.000664   0.000164
	  -0.001587  -0.001248   0.000745   0.000029   0.000232   0.001607   0.001345
	  -0.000010  -0.001255   0.001368   0.001562  -0.000223  -0.000033   0.000291
	   0.000751  -0.000647  -0.002179  -0.000816  -0.000573  -0.000747   0.001585
	   0.000902  -0.000140   0.000664   0.000256  -0.000006   0.000082  -0.000214
	

Optimizer: Optimization complete!


-1998.4081186409126

In [ ]:
print('### Clear constraints')
psi4.p4util.prepare_options_for_set_options()

### Clear constraints


{'SCF__INTS_TOLERANCE': 1e-12}

In [ ]:
print(urea.save_string_xyz())

40

 C   -0.501964777229    0.006131714520    0.079854976194
 O    0.713130575316    0.004953109181    0.065157278319
 N   -1.332947713676    1.169101959420    0.265809112041
 N   -1.339408793168   -1.155037347193   -0.085886715755
 H   -2.238051563563    1.089701419330   -0.205236277162
 H   -2.233451743932   -1.073457130586    0.405376171482
 C   -0.793180715864    2.501337693578    0.223567161277
 C   -1.579200831254    3.529455499377   -0.295534315827
 C    0.464040438176    2.801760563380    0.749874728482
 C   -1.113235533026    4.834155690393   -0.293290110874
 H   -2.561169856195    3.309014675237   -0.694408301841
 C    0.920431662556    4.110130517620    0.737509808969
 H    1.080583818463    2.015805922149    1.163328060995
 C    0.140536037502    5.133275539530    0.217410497887
 H    0.501526462256    6.154165855872    0.221947788712
 C   -0.802127576246   -2.488530190832   -0.056681570074
 C   -1.578470523483   -3.514932830175    0.480232016759
 C    0.441930356568   -2.7

In [ ]:
show_3D(urea)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

#### 正確なヘッセ行列を求める

In [ ]:
# 異なる値用に同じ初期構造で分子を3つ準備
psi4.set_output_file('full_hess_every.log')
hcho = psi4.geometry('''
0 1
 C                 -0.70124482    0.39419087    0.00000000
 O                  0.52607218    0.39419087    0.00000000
 H                 -1.29338982    1.33359487    0.00000000
 H                 -1.29338982   -0.54521313    0.00003900
 ''')

hcho1 = hcho.clone()
hcho2 = hcho.clone()

In [ ]:
for (i, mol) in zip([-1, 0, 1], [hcho, hcho1, hcho2]):
    psi4.core.clean_options()
    psi4.set_options({'FULL_HESS_EVERY': i})
    time = record_optimization(mol=mol, theory='hf/3-21g')
    print(f'FULL_HESS_EVERY = {i}: {time: .2f} sec')

INFO:psi4.optking.optwrapper:Creating a UserComputer
INFO:psi4.optking.optwrapper:
    			-----------------------------------------

    			 OPTKING 3.0: for geometry optimizations 

    			     By R.A. King, Bethel University     

    			        with contributions from          

    			    A.V. Copan, J. Cayton, A. Heide      

    			-----------------------------------------

    
INFO:psi4.optking.optwrapper:
		 -- Optimization Parameters --
	accept_symmetry_breaking       =           False
	alg_geom_maxiter               =              50
	bt_dx_conv                     =           1e-07
	bt_dx_rms_change_conv          =           1e-12
	bt_max_iter                    =              25
	cart_hess_read                 =           False
	consecutive_backsteps_allowed  =               0
	conv_max_DE                    =           1e-06
	conv_max_disp                  =          0.0012
	conv_max_force                 =          0.0003
	conv_rms_disp                  =              -1

Optimizer: Optimization complete!
FULL_HESS_EVERY = -1:  5.02 sec


INFO:psi4.optking.optwrapper:Creating a UserComputer
INFO:psi4.optking.optwrapper:
    			-----------------------------------------

    			 OPTKING 3.0: for geometry optimizations 

    			     By R.A. King, Bethel University     

    			        with contributions from          

    			    A.V. Copan, J. Cayton, A. Heide      

    			-----------------------------------------

    
INFO:psi4.optking.optwrapper:
		 -- Optimization Parameters --
	accept_symmetry_breaking       =           False
	alg_geom_maxiter               =              50
	bt_dx_conv                     =           1e-07
	bt_dx_rms_change_conv          =           1e-12
	bt_max_iter                    =              25
	cart_hess_read                 =           False
	consecutive_backsteps_allowed  =               0
	conv_max_DE                    =           1e-06
	conv_max_disp                  =          0.0012
	conv_max_force                 =          0.0003
	conv_rms_disp                  =              -1

INFO:psi4.driver.driver:Return gradient(): -113.22174134556423
INFO:psi4.driver.driver:[[-0.00086223 -0.00000084  0.00000000]
 [ 0.00262448  0.00000030  0.00000000]
 [-0.00088112  0.00000027 -0.00122908]
 [-0.00088112  0.00000027  0.00122908]]
INFO:psi4.optking.history:	Performing BFGS update.
INFO:psi4.optking.history:	Using 1 previous steps for update.
INFO:psi4.optking.history:	Steps to be used in Hessian update:  1

INFO:psi4.optking.history:	Updated Hessian (in au) 
 	   0.857261   0.045375   0.045375   0.020802   0.020802  -0.041604  -0.000003
	   0.000002  -0.000002
	   0.045375   0.316359   0.013035   0.006265  -0.014606   0.008341  -0.000001
	   0.000001  -0.000001
	   0.045375   0.013035   0.316359  -0.014606   0.006265   0.008341  -0.000001
	   0.000001  -0.000001
	   0.020802   0.006265  -0.014606   0.141145  -0.088302  -0.052843  -0.000001
	   0.000002   0.000003
	   0.020802  -0.014606   0.006265  -0.088302   0.141145  -0.052843  -0.000001
	  -0.000003  -0.000002
	  -0.04

Optimizer: Optimization complete!
FULL_HESS_EVERY = 0:  4.37 sec


INFO:psi4.optking.optwrapper:Creating a UserComputer
INFO:psi4.optking.optwrapper:
    			-----------------------------------------

    			 OPTKING 3.0: for geometry optimizations 

    			     By R.A. King, Bethel University     

    			        with contributions from          

    			    A.V. Copan, J. Cayton, A. Heide      

    			-----------------------------------------

    
INFO:psi4.optking.optwrapper:
		 -- Optimization Parameters --
	accept_symmetry_breaking       =           False
	alg_geom_maxiter               =              50
	bt_dx_conv                     =           1e-07
	bt_dx_rms_change_conv          =           1e-12
	bt_max_iter                    =              25
	cart_hess_read                 =           False
	consecutive_backsteps_allowed  =               0
	conv_max_DE                    =           1e-06
	conv_max_disp                  =          0.0012
	conv_max_force                 =          0.0003
	conv_rms_disp                  =              -1

INFO:psi4.driver.driver:Return gradient(): -113.22174134566245
INFO:psi4.driver.driver:[[-0.00086222 -0.00000084  0.00000000]
 [ 0.00262451  0.00000030  0.00000000]
 [-0.00088115  0.00000027 -0.00122910]
 [-0.00088115  0.00000027  0.00122910]]
INFO:psi4.driver.task_planner:PLANNING Atomic:  keywords={'FULL_HESS_EVERY': 1, 'SCF__GUESS': 'READ', 'SCF__INTS_TOLERANCE': 1e-12, 'function_kwargs': {}}
INFO:psi4.driver.driver:Compute hessian(): method=hf, basis=3-21g, molecule=default, nre=31.45268001447735
INFO:psi4.driver.driver:Return hessian(): -113.22174134566245
INFO:psi4.driver.driver:[[ 1.13485452  0.00000929 -0.00000000 -0.87894652 -0.00000739  0.00000000 -0.12795406 -0.00000095 -0.10384907
  -0.12795406 -0.00000095  0.10384908]
 [ 0.00000929  0.21309068 -0.00000000 -0.00000767 -0.06905129  0.00000000 -0.00000081 -0.07201970 -0.00000138
  -0.00000081 -0.07201970  0.00000138]
 [-0.00000000 -0.00000000  0.67498645  0.00000000  0.00000000 -0.11556255 -0.10163277 -0.00000161 -0.27971195


Optimizer: Optimization complete!
FULL_HESS_EVERY = 1:  11.35 sec
